In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import string
import re

In [2]:
from sklearn import decomposition
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
#read data from kaggle
train = pd.read_csv('../input/train.tsv', sep='\t')
test = pd.read_csv('../input/test.tsv', sep='\t')

In [3]:
#read data for local machine
curr_dir= os.getcwd()
train = pd.read_csv(curr_dir+"//Dataset/train.tsv", sep='\t')
test = pd.read_csv(curr_dir+"//Dataset/test.tsv",sep='\t')

In [42]:
train.head()

,id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3,is_train
0,0,MLB Cincinnati Reds T Shirt Size XL,3,NaN,10.0,1,No description yet,Men,Tops,T-shirts,1
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,1
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,1
3,3,Leather Horse Statues,1,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,1
4,4,24K GOLD plated rose,1,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,1


In [47]:
train.loc[train['id']==3217]

,id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3,is_train
3217,3217,Free to be wild bra,3,Lululemon,49.0,0,Lululemon free to be wild bra sz 4. VEUC. Last...,Women,Athletic Apparel,Sports Bras,1


In [5]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [6]:
def cat_split(row):
    try:
        text = row
        txt1, txt2, txt3 = text.split('/')
        return txt1, txt2, txt3
    except:
        return np.nan, np.nan, np.nan

train["cat_1"], train["cat_2"], train["cat_3"] = zip(*train.category_name.apply(lambda val: cat_split(val)))
test["cat_1"], test["cat_2"], test["cat_3"] = zip(*test.category_name.apply(lambda val: cat_split(val)))


In [7]:
train.info()
print("------------------------------------------")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 11 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
cat_1                1471819 non-null object
cat_2                1471819 non-null object
cat_3                1471819 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 124.4+ MB
------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 10 columns):
test_id              693359 non-null int64
name                 693359 non-null object
item_condition_id    693359 non-null int64
category_name        690301 non-null object
bra

In [8]:
print(train.isnull().sum())
print ("---------------------------")
test.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          4
cat_1                 10716
cat_2                 10716
cat_3                 10716
dtype: int64
---------------------------


test_id                   0
name                      0
item_condition_id         0
category_name          3058
brand_name           295525
shipping                  0
item_description          0
cat_1                  5104
cat_2                  5104
cat_3                  5104
dtype: int64

In [9]:
train.drop(['category_name'], axis=1, inplace = True)
test.drop(['category_name'], axis=1, inplace = True)

In [10]:
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})

In [11]:
train['is_train'] = 1
test['is_train'] = 0

In [281]:
train_test_combine = pd.concat([train.drop(['price'],axis =1),test],axis = 0)

In [282]:
train_test_combine.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
0,0,MLB Cincinnati Reds T Shirt Size XL,3,NaN,1,No description yet,Men,Tops,T-shirts,1
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,1
2,2,AVA-VIV Blouse,1,Target,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,1
3,3,Leather Horse Statues,1,NaN,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,1
4,4,24K GOLD plated rose,1,NaN,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,1


In [283]:
train_test_combine['cat_3'].describe()

count                     2160074
unique                        876
top       Pants, Tights, Leggings
freq                        88077
Name: cat_3, dtype: object

In [284]:
#train_test_combine = train_test_combine.drop(['cat_3'],axis=1)

In [285]:
train_test_combine = train_test_combine.sort_values("cat_1")

In [286]:
cat_1_list = list(train_test_combine['cat_1'].unique())

In [287]:
cat_1_list

['Beauty',
 'Electronics',
 'Handmade',
 'Home',
 'Kids',
 'Men',
 'Other',
 'Sports & Outdoors',
 'Vintage & Collectibles',
 'Women',
 nan]

In [288]:
df_1 = train_test_combine[train_test_combine['cat_1'] == 'Beauty']
df_2 = train_test_combine[train_test_combine['cat_1'] == 'Electronics']
df_3 = train_test_combine[train_test_combine['cat_1'] == 'Handmade']
df_4 = train_test_combine[train_test_combine['cat_1'] == 'Home']
df_5 = train_test_combine[train_test_combine['cat_1'] == 'Kids']
df_6 = train_test_combine[train_test_combine['cat_1'] == 'Other']
df_7 = train_test_combine[train_test_combine['cat_1'] == 'Sports & Outdoors']
df_8 = train_test_combine[train_test_combine['cat_1'] == 'Vintage & Collectibles']
df_9 = train_test_combine[train_test_combine['cat_1'] == 'Women']
df_10 = train_test_combine[train_test_combine['cat_1'].isnull()]

In [289]:
df_1.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
691450,691450,Ulta HD soft focus foundation,3,Ulta,0,Not my color. LIGHT COOL Used a tiny dot of th...,Beauty,Makeup,Face,1
3217,3217,Tartelette in bloom,3,Tarte,1,Only used twice. It is 100% authentic.,Beauty,Makeup,Makeup Palettes,0
974292,974292,❤LIQUIDATING! TOO FACED MELTED LIPPY SET,1,Too Faced,1,PLEASE READ MY BIO BEFORE BUYING FROM MY CLOSE...,Beauty,Makeup,Lips,1
315086,315086,Travel Size Setting Powder,1,Dermablend,1,This is a loose translucent powder that sets m...,Beauty,Makeup,Face,1
315085,315085,4 Too Faced Lipstick bundle,4,Too Faced,0,You get all 4. Usage is a clearly as shown. I ...,Beauty,Makeup,Lips,1


In [290]:
def work_on_each_dataframe(df_w):
    #one hot encoding on one dataset
    df_w = pd.get_dummies(df_w, columns=["cat_1"])
    df_w = pd.get_dummies(df_w, columns=["cat_2"])
    df_w = pd.get_dummies(df_w, columns=["cat_3"])
    
    df_w.name = df_w.name.astype('category')
    #df_w.brand_name = df_w.brand_name.astype('category')
    
    #label Encoding
    #df_w.brand_name = df_w.brand_name.cat.codes
    df_w.name = df_w.name.cat.codes

    df_w = df_w.drop(['item_description'],axis = 1)
    
    return df_w

In [291]:
df_1 = work_on_each_dataframe(df_1)
df_2 = work_on_each_dataframe(df_2)
df_3 = work_on_each_dataframe(df_3)
df_4 = work_on_each_dataframe(df_4)
df_5 = work_on_each_dataframe(df_5)
df_6 = work_on_each_dataframe(df_6)
df_7 = work_on_each_dataframe(df_7)
df_8 = work_on_each_dataframe(df_8)
df_9 = work_on_each_dataframe(df_9)
df_10 = work_on_each_dataframe(df_10)

In [292]:
df_1.shape

(305093, 60)

In [293]:
def model_train(df_m):
    df_test= df_m.loc[df_m['is_train']==0]
    df_train= df_m.loc[df_m['is_train']==1]
    
    df_test = df_test.drop(['is_train'],axis=1)
    df_train = df_train.drop(['is_train'],axis=1)
    
    df_train['price'] = train.price
    #df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)
    return df_train,df_test

In [294]:
df_1_train, df_1_test = model_train(df_1)
df_2_train, df_2_test = model_train(df_2)
df_3_train, df_3_test = model_train(df_3)
df_4_train, df_4_test = model_train(df_4)
df_5_train, df_5_test = model_train(df_5)
df_6_train, df_6_test = model_train(df_6)
df_7_train, df_7_test = model_train(df_7)
df_8_train, df_8_test = model_train(df_8)
df_9_train, df_9_test = model_train(df_9)
df_10_train, df_10_test = model_train(df_10)

In [299]:
df_1_test.head()

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo & Conditioner Sets,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women
3217,3217,204707,3,Tarte,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
390160,390160,42799,1,NaN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
390162,390162,50420,1,NaN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
105204,105204,199998,1,NaN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
390177,390177,174877,1,NaN,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [300]:
df_1_train.head()

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price
691450,691450,213645,3,Ulta,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.0
974292,974292,239965,1,Too Faced,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.0
315086,315086,211153,1,Dermablend,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,10.0
315085,315085,14670,4,Too Faced,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,11.0
1174061,1174061,55214,1,NaN,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0


In [ ]:
'''
#PCA

#convert it to numpy arrays
X=df_train.values

#Scaling the values
X = scale(X)

pca = PCA(n_components=44)

pca.fit(X)

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

'''

In [304]:
'''
brand_group: Dataframe comprising mean of each brand
brand_group1: Training Dataset, in which we will append the brand_weight
brand_group11: Testing Dataset, in which we will append the brand_weight
brand_group2: a temperory variable for train data
brand_group22: a temperory variable for test data 
'''
def brand_weight_train(df_train1):
    brand_group = df_train1.groupby(['brand_name'])['price'].mean().to_frame()
    brand_group['brand_name'] =brand_group.index
    i=0
    for index1,row1 in brand_group.iterrows():
        if i ==0:
            brand_group1 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            brand_group1.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            
            #brand_group11 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            #brand_group11.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            i=1
        else:
            brand_group2 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            brand_group2.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            brand_group1 = brand_group1.append(brand_group2)
            
            #brand_group22 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            #brand_group22.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            #brand_group11 = brand_group11.append(brand_group22)
        
    
    
    return brand_group1#, brand_group11


In [307]:
def brand_weight_test(df_train1,df_test1):
    brand_group = df_train1.groupby(['brand_name'])['price'].mean().to_frame()
    brand_group['brand_name'] =brand_group.index
    i=0
    for index1,row1 in brand_group.iterrows():
        if i ==0:
            #brand_group1 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            #brand_group1.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            
            brand_group11 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            brand_group11.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            i=1
        else:
            #brand_group2 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            #brand_group2.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            #brand_group1 = brand_group1.append(brand_group2)
            
            brand_group22 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
            brand_group22.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
            brand_group11 = brand_group11.append(brand_group22)
        
    
    
    return brand_group11#, brand_group11


In [308]:
df_1_test = brand_weight_test(df_1_train,df_1_test)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ValueError: cannot set a frame with no defined index and a scalar

In [305]:
df_1_train = brand_weight_train(df_1_train)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [306]:
df_1_train

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price,brand_weight
808964,808964,179501,1,% Pure,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,17.0,16.344262
1039740,1039740,100383,1,% Pure,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,10.0,16.344262
307876,307876,179911,1,% Pure,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,15.0,16.344262
817119,817119,103321,3,% Pure,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10.0,16.344262
1171534,1171534,80797,1,% Pure,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,19.0,16.344262
298165,298165,73241,1,% Pure,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,14.0,16.344262
302086,302086,4268,2,% Pure,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,18.0,16.344262
975602,975602,1859,1,% Pure,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,15.0,16.344262
606431,606431,100383,1,% Pure,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,10.0,16.344262
601624,601624,4185,3,% Pure,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,24.0,16.344262


In [301]:
df_1_train

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price
691450,691450,213645,3,Ulta,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.0
974292,974292,239965,1,Too Faced,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.0
315086,315086,211153,1,Dermablend,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,10.0
315085,315085,14670,4,Too Faced,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,11.0
1174061,1174061,55214,1,NaN,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0
1395559,1395559,60478,1,Clinique,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.0
716984,716984,107670,3,Kylie Cosmetics,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,46.0
809896,809896,153250,1,NaN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.0
1278477,1278477,164474,1,L'Oreal,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.0
1258581,1258581,51098,1,NaN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,15.0


In [137]:
def Random_forest(df_train,df_test):
    x_train,y_train = df_train.drop(['price'],axis =1),df_train.price
    m = RandomForestRegressor(n_jobs=-1,min_samples_leaf=3,n_estimators=200)
    m.fit(x_train, y_train)
    print(m.score(x_train,y_train))

In [138]:
Random_forest(df_1_train,df_1_test)
Random_forest(df_2_train,df_2_test)
Random_forest(df_3_train,df_3_test)
Random_forest(df_4_train,df_4_test)
Random_forest(df_5_train,df_5_test)
Random_forest(df_6_train,df_6_test)
Random_forest(df_7_train,df_7_test)
Random_forest(df_8_train,df_8_test)
Random_forest(df_9_train,df_9_test)
Random_forest(df_10_train,df_10_test)

0.541851630146
0.786872725929
0.570298054713
0.63408717858
0.658799071614
0.549974740029


KeyboardInterrupt: 

In [ ]:
preds = m.predict(df_test)

In [ ]:
preds = pd.Series(np.exp(preds))

In [ ]:
type(preds)

In [ ]:
submit = pd.concat([df_test.id,preds],axis=1)

In [ ]:
submit.columns = ['test_id','price']